In [1]:
!pip install transformers langchain textract 

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 24.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 45.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━

In [2]:
!pip install transformers
!pip install langchain
!pip install torch
!pip install tensorflow
!pip install langchain_community

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a v

In [3]:
from transformers import BartForConditionalGeneration, BartTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
model = BartForConditionalGeneration.from_pretrained(
        'facebook/bart-large-cnn')

tokenizer = BartTokenizer.from_pretrained(
            'facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
import textract

def extract_text(file_path):
    try:
        text = textract.process(file_path)
        return text.decode('utf-8')
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None

In [21]:
def summarize(text, maxSummarylength):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors = "pt",
                              max_length = 1024, truncation=True)

    summary_ids = model.generate(inputs,
                                 max_length = maxSummarylength,
                                 min_length = int(maxSummarylength/5),
                                 length_penalty = 10.0,
                                 num_beams = 4,
                                 early_stopping = True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens = True)

    return summary

def split_text_into_pieces(text, max_tokens=1024, use_sentence_boundary=False):
    """
    Splits the text into pieces based on the max_tokens, optionally considering sentence boundaries.
    
    Parameters:
    text (str): The text to be split.
    max_tokens (int): Maximum tokens per piece.
    use_sentence_boundary (bool): Whether to split at sentence boundaries.
    
    Returns:
    List[str]: A list of text pieces.
    """
    if use_sentence_boundary:
        sentences = text.split('. ')
        pieces = []
        current_piece = ''
        for sentence in sentences:
            if len(tokenizer.tokenize(current_piece + sentence)) <= max_tokens:
                current_piece += sentence + '. '
            else:
                pieces.append(current_piece.strip())
                current_piece = sentence + '. '
        if current_piece:
            pieces.append(current_piece.strip())
        return pieces
    else:
        return [text[i:i+max_tokens] for i in range(0, len(text), max_tokens)]

def recursive_summarize(text, max_length=1024, recursion_level=0, detail_level=3):
    """
    Recursive summarization function that provides a concise yet detailed summary.
    
    Parameters:
    text (str): The text to be summarized.
    max_length (int): The maximum token length for each chunk.
    recursion_level (int): Current recursion level (used internally for tracking depth).
    detail_level (int): Level of detail for the final summary. Lower values produce more detailed summaries.
    
    Returns:
    str: The final summary after recursive summarization.
    """
    recursion_level += 1
    print("######### Recursion level: ", recursion_level, "\n\n######### ")

    tokens = tokenizer.tokenize(text)
    expected_count_of_chunks = len(tokens) / max_length
    max_length = int(len(tokens) / expected_count_of_chunks) + 2

    # Break the text into pieces based on max_length, considering sentence boundaries
    pieces = split_text_into_pieces(text, max_tokens=max_length, use_sentence_boundary=True)

    print("Number of pieces: ", len(pieces))

    # Summarize each piece
    summaries = []
    for k, piece in enumerate(pieces):
        print("****************************************************")
        print("Piece:", k + 1, " out of ", len(pieces), "pieces")
        print(piece, "\n")
        
        # Adjust summary length based on detail level
        summary_length = int(max_length / detail_level)
        summary = summarize(piece, maxSummarylength=summary_length)
        
        print("SUMMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    # Concatenate the summaries and tokenize again
    concatenated_summary = ' '.join(summaries)
    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursion_level=recursion_level,
                                   detail_level=detail_level)
    else:
        # Perform a final summarization if necessary
        final_summary = concatenated_summary
        if len(pieces) > 1:
            final_summary = summarize(concatenated_summary,
                                      maxSummarylength=int(max_length / detail_level * 2))

        return final_summary


In [9]:
def split_text_into_pieces(text, max_tokens=1024, use_sentence_boundary=False):
    """
    Splits the text into pieces based on the max_tokens, optionally considering sentence boundaries.
    
    Parameters:
    text (str): The text to be split.
    max_tokens (int): Maximum tokens per piece.
    use_sentence_boundary (bool): Whether to split at sentence boundaries.
    
    Returns:
    List[str]: A list of text pieces.
    """
    if use_sentence_boundary:
        sentences = text.split('. ')
        pieces = []
        current_piece = ''
        for sentence in sentences:
            if len(tokenizer.tokenize(current_piece + sentence)) <= max_tokens:
                current_piece += sentence + '. '
            else:
                pieces.append(current_piece.strip())
                current_piece = sentence + '. '
        if current_piece:
            pieces.append(current_piece.strip())
        return pieces
    else:
        return [text[i:i+max_tokens] for i in range(0, len(text), max_tokens)]

In [24]:
def recursive_summarize(text, max_length=1024, recursion_level=0, detail_level=3):
    """
    Recursive summarization function that provides a concise yet detailed summary.
    
    Parameters:
    text (str): The text to be summarized.
    max_length (int): The maximum token length for each chunk.
    recursion_level (int): Current recursion level (used internally for tracking depth).
    detail_level (int): Level of detail for the final summary. Lower values produce more detailed summaries.
    
    Returns:
    str: The final summary after recursive summarization.
    """
    recursion_level += 1

    tokens = tokenizer.tokenize(text)
    expected_count_of_chunks = len(tokens) / max_length
    max_length = int(len(tokens) / expected_count_of_chunks) + 2

    # Break the text into pieces based on max_length, considering sentence boundaries
    pieces = split_text_into_pieces(text, max_tokens=max_length, use_sentence_boundary=True)

    # Summarize each piece
    summaries = []
    for piece in pieces:
        # Adjust summary length based on detail level
        summary_length = int(max_length / detail_level)
        summary = summarize(piece, summary_length)
        summaries.append(summary)

    # Concatenate the summaries and tokenize again
    concatenated_summary = ' '.join(summaries)
    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated summary is too long, repeat the process
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursion_level=recursion_level,
                                   detail_level=detail_level)
    else:
        # Perform a final summarization if necessary
        final_summary = concatenated_summary
        if len(pieces) > 1:
            final_summary = summarize(concatenated_summary,
                                      maxSummarylength=int(max_length / detail_level * 2))

        return final_summary

In [25]:
document = extract_text("/kaggle/input/dbms-da/DBMS Unit 5 Tesseract.pdf")

if document:
    summary = recursive_summarize(document, max_length=1024, detail_level=3)
    print("Concise and Detailed Summary:")
    print(summary)
else:
    print("Failed to extract text from the document.")

Extracted text from /kaggle/input/dbms-da/DBMS Unit 5 Tesseract.pdf: b'DATABASE MANAGEMENT SYSTEMS \n\nUNIT V \xe2\x80\x93 TOPIC 1 \n\nTriggers \n\nA trigger is a procedure which is auto'...
Concise and Detailed Summary:
A trigger is a stored database object that is automatically executed or fired on a DML command. A procedure is a subroutine or a subprogram in the language of the database. Triggers are used to invoke procedures by using other triggers, such as Java, Python, PHP, etc. The Data Controlling Language (DCL) helps users to retrieve, modify and delete data stored on the server. DCL commands can be used to grant or revoke user permissions and privileges as per the administrator’s needs. The DBA is the focal point for data/user interaction in an organization. The technical aspects of the DBA's job are rooted in the following areas of operation: design, development, implementation, and maintenance.


In [12]:
!pip install pyngrok flask_ngrok

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [37]:
!pip install flask_cors

  Using cached Flask_Cors-4.0.1-py2.py3-none-any.whl.metadata (5.5 kB)
Using cached Flask_Cors-4.0.1-py2.py3-none-any.whl (14 kB)
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [13]:
!ngrok authtoken "YOUR AUTHTOKEN KEY"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [14]:
from pyngrok import ngrok

# Connect to the local Flask server running on port 5000
public_url = ngrok.connect(addr="5000")
print('* ngrok tunnel "{}" -> "http://127.0.0.1:5000"'.format(public_url))


* ngrok tunnel "NgrokTunnel: "https://1d60-34-32-223-176.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"


In [15]:
%mkdir templates -p

In [28]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document Summarizer</title>
</head>
<body>
    <h1>Document Summarizer</h1>
    <form action="/" method="POST" enctype="multipart/form-data">
        <label for="file">Upload a file:</label><br>
        <input type="file" id="file" name="file"><br><br>
        <input type="submit" value="Summarize">
    </form>
    
    {% if summary_text %}
    <h2>Summary:</h2>
    <textarea id="summary_text" rows="10" cols="80">{{ summary_text }}</textarea><br><br>
    <p>Generation Time: {{ generation_time }} seconds</p>
    {% else %}
    <p>No summary generated yet.</p>
    {% endif %}
</body>
</html>


Overwriting templates/index.html


In [36]:
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import os
import time
import textract
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Allowed file extensions
ALLOWED_EXTENSIONS = {'pdf', 'docx', 'txt'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def extract_text(file_path):
    try:
        text = textract.process(file_path)
        print(f"Extracted text from {file_path}: {text[:100]}...")  # Log the first 100 characters of extracted text
        return text.decode('utf-8')
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        # Process the uploaded file or the input text
        if 'file' in request.files:
            file = request.files['file']
            if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file_path = os.path.join(app.root_path, filename)
                print(f"Saving file to {file_path}")  # Log file saving
                file.save(file_path)
                input_text = extract_text(file_path)
                os.remove(file_path)  # Clean up the file after processing
                if input_text is None:
                    return render_template('index.html', summary_text='Failed to extract text from the file.', generation_time=0)
            else:
                return render_template('index.html', summary_text='Invalid file type.', generation_time=0)
        else:
            input_text = request.form.get('input_text', '')

        summary, generation_time = get_summary(input_text)
        return render_template('index.html', summary_text=summary, generation_time=generation_time)
    return render_template('index.html')

@app.route('/summarize', methods=['POST'])
def summarize_route():
    try:
        print("Received request for summarization")  # Log when a request is received

        # Check if the request contains a file
        if 'file' in request.files:
            print("File found in request")  # Log when a file is found
            file = request.files['file']
            if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file_path = os.path.join(app.root_path, filename)
                print(f"Saving file to {file_path}")  # Log file saving
                file.save(file_path)
                input_text = extract_text(file_path)
                os.remove(file_path)  # Clean up the file after processing
                if input_text is None:
                    raise ValueError("Failed to extract text from the file.")
            else:
                print("Invalid file type")  # Log invalid file type
                return jsonify({
                    'input_text': '',
                    'summary_text': 'Invalid file type.',
                    'generation_time': 0
                })
        else:
            print("No file found, checking for input_text in JSON")  # Log no file found
            input_text = request.form.get('input_text', '')

        # Use the recursive summarization method
        summary, generation_time = get_summary(input_text)
        return jsonify({
            'input_text': input_text,
            'summary_text': summary,
            'generation_time': generation_time
        })
    except Exception as e:
        print(f"Error occurred: {e}")  # Log any errors that occur
        return jsonify({
            'input_text': '',
            'summary_text': f'An error occurred: {str(e)}',
            'generation_time': 0
        })

def get_summary(text):
    try:
        print("Generating summary...")  # Log before generating summary
        start_time = time.time()
        summary = recursive_summarize(text, max_length=1024, detail_level=3)  # Call to recursive summarization
        end_time = time.time()
        generation_time = round(end_time - start_time, 2)
        print(f"Generated summary: {summary[:100]}...")  # Log the first 100 characters of the summary
        return summary, generation_time
    except Exception as e:
        print(f"An error occurred during summary generation: {e}")
        return f"An error occurred: {str(e)}", 0

# Get a public URL
public_url = ngrok.connect(5000)
print(f" * Running on {public_url}")

if __name__ == '__main__':
    app.run()


 * Running on NgrokTunnel: "https://e71e-34-32-223-176.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from pyngrok import ngrok
import os
import time
import textract
from werkzeug.utils import secure_filename

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}, supports_credentials=True)  # Enable CORS for all origins

# Allowed file extensions
ALLOWED_EXTENSIONS = {'pdf', 'docx', 'txt'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def extract_text(file_path):
    try:
        text = textract.process(file_path)
        return text.decode('utf-8')
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        if 'file' in request.files:
            file = request.files['file']
            if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file_path = os.path.join(app.root_path, filename)
                file.save(file_path)
                input_text = extract_text(file_path)
                os.remove(file_path)  # Clean up the file after processing
                if input_text is None:
                    return render_template('index.html', summary_text='Failed to extract text from the file.', generation_time=0)
            else:
                return render_template('index.html', summary_text='Invalid file type.', generation_time=0)
        else:
            input_text = request.form.get('input_text', '')

        summary, generation_time = get_summary(input_text)
        return render_template('index.html', summary_text=summary, generation_time=generation_time)
    return render_template('index.html')

@app.route('/summarize', methods=['POST'])
def summarize_route():
    print("Received request")  # Debugging log
    try:
        if 'file' in request.files:
            print("File found")  # Debugging log
            file = request.files['file']
            if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file_path = os.path.join(app.root_path, filename)
                file.save(file_path)
                input_text = extract_text(file_path)
                os.remove(file_path)
                if input_text is None:
                    raise ValueError("Failed to extract text from the file.")
            else:
                print("Invalid file type")  # Debugging log
                return jsonify({'input_text': '', 'summary_text': 'Invalid file type.', 'generation_time': 0})
        else:
            print("No file, checking for input_text")  # Debugging log
            input_text = request.form.get('input_text', '')

        summary, generation_time = get_summary(input_text)
        return jsonify({'input_text': input_text, 'summary_text': summary, 'generation_time': generation_time})
    except Exception as e:
        print(f"Error: {e}")  # Debugging log
        return jsonify({'input_text': '', 'summary_text': f'An error occurred: {str(e)}', 'generation_time': 0})

def get_summary(text):
    try:
        print("Generating summary...")  # Log before generating summary
        start_time = time.time()
        summary = recursive_summarize(text, max_length=1024, detail_level=3)
        end_time = time.time()
        generation_time = round(end_time - start_time, 2)
        print(f"Generated summary: {summary[:100]}...")  # Log the first 100 characters of the summary
        print("Summaryy sent")
        return summary, generation_time
    except Exception as e:
        print(f"An error occurred during summary generation: {e}")
        return f"An error occurred: {str(e)}", 0


# Get a public URL
public_url = ngrok.connect(5000)
print(f" * Running on {public_url}")

if __name__ == '__main__':
    app.run()


 * Running on NgrokTunnel: "https://bc02-34-32-223-176.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
Received request
File found
Generating summary...
Generated summary: The cat command in Linux is a short form for "concatenate" It is one of the most frequently used com...
